<a href="https://colab.research.google.com/github/zelihami/NLP-examples/blob/main/Bert_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/content/nlpfinalyt.csv")

In [ ]:
df

,metin,label
0,20 dk 1 GB internet 500 mb sadece kaşar turkce...,olumsuz
1,Ayrıca turkcell superonline reklamı kadar da k...,olumsuz
2,Turkcell çok pahalı ya,olumsuz
3,Turkcell Kaş'ta internetin cekmiyor,olumsuz
4,Turkcell'in Allah belası versin demek isterdim...,olumsuz
...,...,...
13826,@Buragl111 turkcell kazık takıl sen bimcell le...,olumsuz
13827,@Turkcell merhaba turkcell.20 senelik dostlugu...,olumlu
13828,@TurkcellHizmet merhaba. Numarami bir baska op...,olumsuz
13829,iyi de Herkes TURKCELL Abonesi Değil ki ??,notr


In [ ]:
df = df[df['label'] != 'notr'].reset_index(drop=True)

In [ ]:
df.isna().sum()

metin    0
label    0
dtype: int64

In [ ]:
df["label"] = df["label"].replace({"olumlu": 1, "olumsuz": 0})

In [ ]:
df

,metin,label
0,20 dk 1 GB internet 500 mb sadece kaşar turkce...,0
1,Ayrıca turkcell superonline reklamı kadar da k...,0
2,Turkcell çok pahalı ya,0
3,Turkcell Kaş'ta internetin cekmiyor,0
4,Turkcell'in Allah belası versin demek isterdim...,0
...,...,...
9168,Ekonomi Haberi: Turkcell'den 1.500 kişiye iş i...,1
9169,@Buragl111 turkcell kazık takıl sen bimcell le...,0
9170,@Turkcell merhaba turkcell.20 senelik dostlugu...,1
9171,@TurkcellHizmet merhaba. Numarami bir baska op...,0


In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")
#model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import torch

In [ ]:
df.to_csv('nlp_son.csv', index=False)

In [ ]:
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
from datasets import load_dataset
my_data=load_dataset("csv",data_files="/content/nlp_son.csv",encoding="UTF-8")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
my_data

DatasetDict({
    train: Dataset({
        features: ['metin', 'label'],
        num_rows: 9173
    })
})

In [ ]:
small_train_dataset=my_data["train"].shuffle(seed=42)

In [ ]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
mydata=small_train_dataset.train_test_split(train_size=0.9,seed=42)

In [ ]:
small_train_dataset = mydata["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = mydata["test"].shuffle(seed=42).select([i for i in list(range(300))])
print(small_train_dataset[0])
print(small_test_dataset[0])

{'metin': '#yIldIztilbeyanlIzdeGildir unutma bunu turkcell pis yahudiler unutmayIn!!! #CareyIldIztilbe @turkcell yazIklar olsun sana !!!', 'label': 0}
{'metin': '@Semocandir Turkcell sim diye birşey buldum onu indirmeye çalışıyorum yarım saattir inmesini bekliyorum hahajajah', 'label': 0}


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["metin"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-128k-uncased") # sınıflandırma yapacağımız için bu base modeli kullanıyoruz

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
from transformers import TrainingArguments, Trainer


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#pip install transformers[torch]

In [ ]:
#pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.0 MB/s eta 0:00:00


In [ ]:
repo_name = "nlpfinalbert0"


In [ ]:
training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=376, training_loss=0.3707299739756483, metrics={'train_runtime': 2840.8141, 'train_samples_per_second': 2.112, 'train_steps_per_second': 0.132, 'total_flos': 118852197038400.0, 'train_loss': 0.3707299739756483, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

<ipython-input-26-6b0938f570ed>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'eval_loss': 0.3191142678260803,
 'eval_accuracy': 0.88,
 'eval_f1': 0.8348623853211008,
 'eval_runtime': 32.9414,
 'eval_samples_per_second': 9.107,
 'eval_steps_per_second': 0.577,
 'epoch': 2.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1705713715.fcb3cd82a140.1898.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/zelihami/nlpfinalbert0/commit/b029dabcff2c5b0d863e1b137bd49b847d0487ed', commit_message='End of training', commit_description='', oid='b029dabcff2c5b0d863e1b137bd49b847d0487ed', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
from transformers import pipeline

sentiment_model = pipeline(model="zelihami/nlpfinalbert0")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/737M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [4]:
sonuc=sentiment_model(["havalar çok ısındı dışarı çıkılmaz artık"])
if (sonuc[0]["label"]=="LABEL_1"):
    print("positive")
elif (sonuc[0]["label"]=="LABEL_0"):
    print("negative")

negative
